## Create the metadata to alias

In this example, we create two Operators that we will subsequently alias.

In [ ]:
from ni.datastore import Client
from ni.measurements.metadata.v1.metadata_store_pb2 import Operator

client = Client()
operatorOne = Operator(operator_name="Jane Doe")
operatorTwo = Operator(operator_name="John Smith")

client.create_operator(operatorOne)
client.create_operator(operatorTwo)

## Create aliases for the desired metadata

Note: The metadata being aliased must have already been created prior to creating the alias.

In [ ]:
client.create_alias("primary_operator", operatorOne)
client.create_alias("secondary_operator", operatorTwo)

print("Aliases created successfully.")

## Reference the alias

You can reference the alias in place of the metadata ID when creating further metadata or publishing.

In [ ]:
from ni.datastore.data import TestResult

test_result = TestResult(
    test_result_name="Test Result with Operator Alias",
    operator_id="secondary_operator"
)

test_result_id = client.create_test_result(test_result)

print("The test results was created successfully using an alias to reference an operator.")


## Verify expected alias usage

In [ ]:
# Retrieve the session and verify which operator was associated with it
retrieved_session = client.get_test_result(test_result_id)
retrieved_operator_id = retrieved_session.operator_id
operator_details = client.get_operator(retrieved_operator_id)

print(f"The name of the operator for the session: {operator_details.operator_name}")

## Get alias

Look up the target information of the alias (i.e. what the alias is an alias for)

In [ ]:
alias = client.get_alias("primary_operator")

print(alias)

## Query all aliases

In [ ]:
aliases = client.query_aliases("")
for alias in aliases:
    print(alias)

## Query aliases with filter

In [ ]:
aliases = client.query_aliases("$filter=name eq 'primary_operator'")
for alias in aliases:
    print(alias)

## Delete alias

Deletes an existing alias on the system. This does not affect the underlying target/metadata to which the alias refers.

In [ ]:
deleted = client.delete_alias("primary_operator")

print(f"Success of deleting 'primary_operator': {deleted}\n")

## Query the aliases again to confirm the alias has been removed
aliases = client.query_aliases("")
for alias in aliases:
    print(alias)

# Try to delete the alias again. This should not succeed since the alias has already been removed.
deleted = client.delete_alias("primary_operator")
print(f"Success of deleting already-removed 'primary_operator': {deleted}")